<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 2

### Exploratory Data Analysis (EDA)

---

Your hometown mayor just created a new data analysis team to give policy advice, and the administration recruited _you_ via LinkedIn to join it. Unfortunately, due to budget constraints, for now the "team" is just you...

The mayor wants to start a new initiative to move the needle on one of two separate issues: high school education outcomes, or drug abuse in the community.

Also unfortunately, that is the entirety of what you've been told. And the mayor just went on a lobbyist-funded fact-finding trip in the Bahamas. In the meantime, you got your hands on two national datasets: one on SAT scores by state, and one on drug use by age. Start exploring these to look for useful patterns and possible hypotheses!

---

This project is focused on exploratory data analysis, aka "EDA". EDA is an essential part of the data science analysis pipeline. Failure to perform EDA before modeling is almost guaranteed to lead to bad models and faulty conclusions. What you do in this project are good practices for all projects going forward, especially those after this bootcamp!

This lab includes a variety of plotting problems. Much of the plotting code will be left up to you to find either in the lecture notes, or if not there, online. There are massive amounts of code snippets either in documentation or sites like [Stack Overflow](https://stackoverflow.com/search?q=%5Bpython%5D+seaborn) that have almost certainly done what you are trying to do.

**Get used to googling for code!** You will use it every single day as a data scientist, especially for visualization and plotting.

#### Package imports

In [ ]:
import numpy as np
import scipy.stats as stats
from scipy.stats import t
from scipy.stats import scoreatpercentile
import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline
plt.style.use('fivethirtyeight')

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings('ignore')


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the `sat_scores.csv` dataset and describe it

---

You should replace the placeholder path to the `sat_scores.csv` dataset below with your specific path to the file.

### 1.1 Load the file with the `csv` module and put it in a Python dictionary

The dictionary format for data will be the column names as key, and the data under each column as the values.

Toy example:
```python
data = {
    'column1':[0,1,2,3],
    'column2':['a','b','c','d']
    }
```

In [ ]:
def print_dictionary(dictname):
    # print it out for checking key by key
    for dic in dictname:
        print(f"\nkey = {dic} and items below:")
        print(dictname[dic])    
        print("-"*80)

csv_scores = "sat_scores.csv"
print(f"\nReading csv file {csv_scores} as dictionary")
try:
    # open the csv file and convert to dictionary
    dict_sat = pd.read_csv(csv_scores,na_filter=False,sep=',').to_dict()

    # print it out for checking the SAT dictionary
    print_dictionary(dict_sat)
    
except:
    print("Unable to read ",csv_scores)
    

### 1.2 Make a pandas DataFrame object with the SAT dictionary, and another with the pandas `.read_csv()` function

Compare the DataFrames using the `.dtypes` attribute in the DataFrame objects. What is the difference between loading from file and inputting this dictionary (if any)?

In [ ]:
# make a dataframe with the SAT dictionary
df_from_dic = pd.DataFrame.from_dict(dict_sat)

print("the data types of the dataframe from dictionary:")
print(df_from_dic.dtypes)
print("\nfirst 5 rows:")
print(df_from_dic.head())
print("-"*50)

# make a dataframe from csv file, turn off the na_filter
csv_scores = "sat_scores.csv"
print(f"\nReading csv file {csv_scores}")
try:
    # follow the same column name as previous dataframe df_from_dic
    df_from_file = pd.read_csv(csv_scores,na_filter=False,names=df_from_dic.columns,skiprows=1,sep=',')
except:
    print(f"something is wrong with reading the file {csv_scores}")

# list all unique values, check for abnormal values
colname = "State"
unique_col_vals = df_from_file[colname].unique()

# And sort by first column since it seems not major problems
unique_col_vals = sorted(unique_col_vals)
print("\t\t",unique_col_vals)

# sort the dataframe by State, preparing for the dictionary keys to be sorted
df_from_file = df_from_file.sort_values(by='State')

# df_from_file => convert to float
df_from_file['Rate'] = df_from_file['Rate'].astype('float')
df_from_file['Verbal'] = df_from_file['Verbal'].astype('float')
df_from_file['Math'] = df_from_file['Math'].astype('float')

print("\nthe data types of the dataframe from file:")
print(df_from_file.dtypes)
print("\nfirst 5 rows:")
print(df_from_file.head())
print("-"*50)


### What is the difference between loading from file and inputting this dictionary (if any)?

In [ ]:
"""
 the difference betweem the two : 
 Rate,Verbal,Math columns in were 'objects' in df_from_dic
 but were 'int64' in df_from_file
 and we changed the numberic columns from int64 to float64
"""
# compare columns with dtypes
print("Comparing for columns for both dataframes:")
print("-"*50)
print(f"Columns\tDictionary\tRead CSV")
for x,v in enumerate(list(df_from_dic.dtypes)):
    colname = df_from_dic.columns[x]
    dictype = df_from_dic[colname].dtypes
    csvtype = df_from_file[colname].dtypes
    print(f"{colname}\t{dictype}\t\t{csvtype}")

print("-"*50)


If you did not convert the string column values to float in your dictionary, the columns in the DataFrame are of type `object` (which are string values, essentially). 

### 1.3 Look at the first ten rows of the DataFrame: what does our data describe?

From now on, use the DataFrame loaded from the file using the `.read_csv()` function.

Use the `.head(num)` built-in DataFrame function, where `num` is the number of rows to print out.

You are not given a "codebook" with this data, so you will have to make some (very minor) inference.

In [ ]:
# cleaning data,remove the row with invalid 
df_sat = df_from_file[ df_from_file.State.str.len()==2].copy() # make a copy
df_sat.dropna(how='all')                                       # remove NA's
df_sat.reset_index(inplace=True)                               # reset index
df_sat.drop(columns='index', inplace=True)                     # drop index columns
df_sat.head()                                                  # first 5 rows

# free up memory
del df_from_dic, df_from_file                                  # free up variables

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Create a "data dictionary" based on the data

---

A data dictionary is an object that describes your data. This should contain the name of each variable (column), the type of the variable, your description of what the variable is, and the shape (rows and columns) of the entire dataset.


**Codebook which describes the data:**
______________________________________________________________________________ 
  - State (object type): the name of a state in the 2 letters format
  - Rate  (float64 type) : the rate of people attending SAT test in a state
  - Verbal (float64 type): the average verbal score of a state
  - Math (float64 type)   : the average math score of a state
  
Rate, Verbal, Math should be float for decimal points when aggregating.

______________________________________________________________________________


In [ ]:
print("Checking the SAT dataframe:")
print("\n# of rows and columns")
print(df_sat.shape)
print("\nList out columns info")
print(df_sat.info())

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Plot the data using seaborn

---

### 3.1 Using seaborn's `distplot`, plot the distributions for each of `Rate`, `Math`, and `Verbal`

Set the keyword argument `kde=False`. This way you can actually see the counts within bins. You can adjust the number of bins to your liking. 

[Please read over the `distplot` documentation to learn about the arguments and fine-tune your chart if you want.](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.distplot.html#seaborn.distplot)

In [ ]:
# To show the figure on top of each bar using ax.text()
# ax is the axis object from plot object
def AddDataLabel(ax):
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
                fontsize=12, color='black', ha='center', va='bottom')
    return

# Plot a seabon distribution bar graph
# input :
#        "data" - data series
#        "title" - Title of the graph
#        "labelx" - Label of the X axis
#        "labely" - Label of the Y axis
def SeabornDistPlot(data,title,labelx,labely):
    sns.set(color_codes=True)   # turn on colour codes
    subplt = plt.subplot(111)   # 1x1 grid and subplot #1
    ax = sns.distplot(data, rug = True, bins =10, 
                      kde=False, ax=subplt, vertical=False)
    ax.legend(loc='upper center')
    ax.set_xlabel(labelx)
    ax.set_ylabel(labely)
    ax.set_title(title)
    AddDataLabel(ax)
    # Unable to fix below warning
    # No handles with labels found to put in legend.
    plt.show(ax)

In [ ]:
# Plot a graph , data take from "Rate" column of the dataframe
SeabornDistPlot(df_sat.Rate, "Rate of SAT test attended in a state", 
                "Index",'# of peoples')

In [ ]:
# Plot a graph , data take from "Verbal" column of the dataframe
SeabornDistPlot(df_sat.Verbal, "Average verbal score of a state", 
                "Index",'Score')

In [ ]:
# Plot a graph , data take from "Math" column of the dataframe
SeabornDistPlot(df_sat.Math, "Average math score of a state", 
                "Index",'Score')

### 3.2 Using seaborn's `pairplot`, show the joint distributions for each of `Rate`, `Math`, and `Verbal`

Explain what the visualization tells you about your data.

[Please read over the `pairplot` documentation to fine-tune your chart.](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.pairplot.html#seaborn.pairplot)

In [ ]:
# Plot a seabon pair-plot graph
# input :
#        "data" - data series
#        "title" - Title of the graph
#        "labelx" - Label of the X axis
#        "labely" - Label of the Y axis
def SeabornPairPlot(data,title,labelx,labely):
    sns.set(color_codes=True)
    ax = sns.pairplot(data)
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
# Plot from entire dataframe for SAT
SeabornPairPlot(df_sat, "Rate of SAT test attended in a state", "Index",'# of peoples')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Plot the data using built-in pandas functions.

---

Pandas is very powerful and contains a variety of nice, built-in plotting functions for your data. Read the documentation here to understand the capabilities:

http://pandas.pydata.org/pandas-docs/stable/visualization.html

### 4.1 Plot a stacked histogram with `Verbal` and `Math` using pandas

In [ ]:
# Plot a stacked histogram 
# input :
#        "data" -   data series of 2 columns
#        "title" -  Title of the graph
#        "labelx" - Label of the X axis
#        "labely" - Label of the Y axis
def StackedHistogram(data,title,labelx,labely):
    fig = plt.figure(1, figsize=(12, 8))
    subplt = plt.subplot(111)
    colors = ['blue', 'orange', 'green']
    data.plot(kind='bar', width=0.6, stacked=True,ax=subplt) 
    plt.legend(loc='upper right')
    plt.xlabel(labelx,fontsize=16)
    plt.ylabel(labely,fontsize=16)
    plt.title(title, fontsize=25)
    plt.show()    

In [ ]:
# Plot a stacked histogram on the Verbal and Math columns
df=df_sat[["Verbal","Math"]]
StackedHistogram(df, "Verbal and Math", "Index",'Score')

### 4.2 Plot `Verbal` and `Math` on the same chart using boxplots

What are the benefits of using a boxplot as compared to a scatterplot or a histogram?

What's wrong with plotting a box-plot of `Rate` on the same chart as `Math` and `Verbal`?

In [ ]:
# Plot a box-plot 
# input :
#        "data" -   data series of 2 columns
#        "title" -  Title of the graph
#        "labelx" - Label of the X axis
#        "labely" - Label of the Y axis
def PltBoxPlot(data,title,labelx,labely):
    data.plot.box()
    plt.xlabel(labelx,fontsize=16)
    plt.ylabel(labely,fontsize=16)
    plt.title(title, fontsize=25)
    plt.legend(loc='upper right')
    plt.show()    
    
# Plot a Seaborn Box-plot
# input :
#        "data" -   data series of 2 columns
#        "title" -  Title of the graph
#        "labelx" - Label of the X axis
#        "labely" - Label of the Y axis
def SeabornBoxPlot(data,orien,title,labelx,labely):
    fig = plt.figure(1, figsize=(12, 8))
    subplt = plt.subplot(111)
    ax = sns.boxplot(data=data, orient=orien, fliersize=5, linewidth=3, notch=True, saturation=0.5, ax=subplt)    
    plt.legend(loc='upper right')
    plt.xlabel(labelx,fontsize=16)
    plt.ylabel(labely,fontsize=16)
    plt.title(title, fontsize=25)
    plt.show()    

In [ ]:
# Rate is not a score, should not in this plot with others
# however just put into one boxplot to see 
# Verbal and Maths very similar but Rate is way below
df = df_sat.loc[:,['Verbal','Math', 'Rate']]
PltBoxPlot(df, 'SAT Score Dataframe', 'Type of Score', 'Score')

In [ ]:
# Verbal and Maths very similar but Maths has long tail
df = df_sat.loc[:,['Verbal','Math']]
PltBoxPlot(df, 'SAT Score Dataframe', 'Type of Score', 'Score')

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 4.3 Plot `Verbal`, `Math`, and `Rate` appropriately on the same boxplot chart

Think about how you might change the variables so that they would make sense on the same chart. Explain your rationale for the choices on the chart. You should strive to make the chart as intuitive as possible. 


In [ ]:
# Standardizing all columns and re-create the boxplot
# We calculate the mean and standard deviation
# and creates a normalised dataframe df_new 
# then plot a seaborn boxplot to show the 3 columns
df = df_sat[['Rate','Verbal', 'Math']]
df_mean = df_sat.mean()
df_std  = df_sat.std() 
df_new = (df - df_mean)/df_std
# Plot a horizontal seaborn box-plot showing  Rate, Verbal, Math
SeabornBoxPlot(df_new,'h','SAT Score Dataframe', 'Type of Score', 'Score')
del df_new, df

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Create and examine subsets of the data

---

For these questions you will practice **masking** in pandas. Masking uses conditional statements to select portions of your DataFrame (through boolean operations under the hood.)

Remember the distinction between DataFrame indexing functions in pandas:

    .iloc[row, col] : row and column are specified by index, which are integers
    .loc[row, col]  : row and column are specified by string "labels" (boolean arrays are allowed; useful for rows)
    .ix[row, col]   : row and column indexers can be a mix of labels and integer indices
    
For detailed reference and tutorial make sure to read over the pandas documentation:

http://pandas.pydata.org/pandas-docs/stable/indexing.html



### 5.1 Find the list of states that have `Verbal` scores greater than the average of `Verbal` scores across states

How many states are above the mean? What does this tell you about the distribution of `Verbal` scores?




In [ ]:
df_mean = df_sat.mean()            # mean value of the columns in SAT dataframe
mean_verbal = float(df_mean[1])    # mean value on the Verbal column
print(f"Average of Verbal score across the states {mean_verbal}\n")

print("List of states having Verbal scores > the average of Verbal scores:")
df_sat[['State','Verbal']][df_sat.Verbal > mean_verbal].sort_values(by="Verbal",ascending=False)

### 5.2 Find the list of states that have `Verbal` scores greater than the median of `Verbal` scores across states

How does this compare to the list of states greater than the mean of `Verbal` scores? Why?

In [ ]:
df_median = df_sat.median()            # median value of the columns in SAT dataframe
median_verbal = float(df_median[1])    # median value on the Verbal column
print(f"Median of Verbal score across the states {median_verbal}\n")

print("List of states having Verbal scores > the average of Verbal scores:")
df_sat[['State','Verbal']][df_sat.Verbal > median_verbal].sort_values(by="Verbal",ascending=False)

### 5.3 Create a column that is the difference between the `Verbal` and `Math` scores

Specifically, this should be `Verbal - Math`.

In [ ]:
df_sat["Verbal - Math"] = (df_sat.Verbal - df_sat.Math)

### 5.4 Create two new DataFrames showing states with the greatest difference between scores

1. Your first DataFrame should be the 10 states with the greatest gap between `Verbal` and `Math` scores where `Verbal` is greater than `Math`. It should be sorted appropriately to show the ranking of states.
2. Your second DataFrame will be the inverse: states with the greatest gap between `Verbal` and `Math` such that `Math` is greater than `Verbal`. Again, this should be sorted appropriately to show rank.
3. Print the header of both variables, only showing the top 3 states in each.

In [ ]:
df_sat[df_sat.Verbal > df_sat.Math].sort_values(by="Verbal - Math",ascending=True).head(10).reset_index()

## 6. Examine summary statistics

---

Checking the summary statistics for data is an essential step in the EDA process!

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.1 Create the correlation matrix of your variables (excluding `State`).

What does the correlation matrix tell you?


In [ ]:
# "Verbal - Math" column not needed for the correlation matrix
if df_sat.shape[1]>4:
    del df_sat["Verbal - Math"]

# find the correlation of the SAT dataframe
print("The correlation matrix for SAT")
sat_corr = df_sat.corr()
print(sat_corr)

# Plot the heatmap graph
sns.heatmap(sat_corr, annot=True)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.2 Use pandas'  `.describe()` built-in function on your DataFrame

Write up what each of the rows returned by the function indicate.

In [ ]:
"""
count  : there were 51 rows of records in the SAT Score
mean   : average value of each columns
std    : standard deviation of each columns
min    : minimum value of each columns
25%    : 25 percentile
50%    : 50 percentile
75%    : 75 percentile
max    : maximum value of each columns
"""
df_sat.describe()

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.3 Assign and print the _covariance_ matrix for the dataset

1. Describe how the covariance matrix is different from the correlation matrix.
2. What is the process to convert the covariance into the correlation?
3. Why is the correlation matrix preferred to the covariance matrix for examining relationships in your data?

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Performing EDA on "drug use by age" data.

---

You will now switch datasets to one with many more variables. This section of the project is more open-ended - use the techniques you practiced above!

We'll work with the "drug-use-by-age.csv" data, sourced from and described here: https://github.com/fivethirtyeight/data/tree/master/drug-use-by-age.

### 7.1

Load the data using pandas. Does this data require cleaning? Are variables missing? How will this affect your approach to EDA on the data?

In [ ]:
csv_drug = "drug-use-by-age.csv"

print(f"\nReading csv file {csv_drug}")
try:
    df_drug = pd.read_csv(csv_drug,encoding='utf-8')
except:
    print(f"something is wrong with reading the file {csv_scores}")
    df_drug = None
if df_drug is not None:
    print("Success!")

In [ ]:
print("Look up for object data type")
print(df_drug.dtypes)

print("\nSome '-' among the  columns")
df_drug.select_dtypes(include="object").head()


In [ ]:
# Look for unused columns
df_drug.tail()

In [ ]:
# remove unused columns such as 'n' 
if df_drug.shape[1]>27:
    del df_drug['n']

# rename the columns
# the frequency columns were actually median
# in the null hypothesis, mean and median were the same
# we rename those fields as mean for this case
column_names = df_drug.columns
column_names = [x.replace('-frequency','_mean') if ('-frequency' in x) else x for x in column_names]
column_names = [x.replace('-','_') if ('-' in x) else x for x in column_names]
print(column_names)
df_drug.columns = column_names

In [ ]:
# age field containing range of values
#df_drug['age'] = [float(x[0:2]) for x in df_drug.age]
df_drug.age.unique()

In [ ]:
# assign age group
def agegroup(x):
    # extract the age value
    if len(x)==5 and x[2]=='-':
        n=int(x[3:6])
    elif len(x)==3 and x[2]=='+':        
        n=int(x[0:2])
    else:
        n=int(x[0:2])

    # Assign age-group 
    if n < 20:
        return 1
    elif n < 30:
        return 2
    elif n < 40:
        return 3
    elif n < 50:
        return 4
    else:
        return 5

df_drug['age_group'] = df_drug['age'].apply(agegroup)

In [ ]:
# check the results
df_drug[['age','age_group']]

In [ ]:
print("These are the columns containing '-' inside:")
[col for col in df_drug.columns if [v for v in df_drug[col].unique() if v=='-'] ]

In [ ]:
# Make possible to assign '0' into a row/column during list comprehension
def setvaluebyloc(colname, colindex):
    df_drug[colname].iloc[colindex] = '0'
    return f"row #{colindex} in {colname} updated."

In [ ]:
# replace '-' with '0' for all 'object' field
# and convert numeric string to float
for col in list(df_drug.select_dtypes(include="object").columns):    
    if col=="age":
        continue
    [setvaluebyloc(col, k) for k in df_drug[col][df_drug[col].str.match('-')].keys() ]
    df_drug[col]=df_drug[col].apply(lambda x: float(x))

In [ ]:
# The _use and _mean columns became float
df_drug.info()    

### 7.2 Do a high-level, initial overview of the data

Get a feel for what this dataset is all about.

Use whichever techniques you'd like, including those from the SAT dataset EDA. The final response to this question should be a written description of what you infer about the dataset.

Some things to consider doing:

- Look for relationships between variables and subsets of those variables' values
- Derive new features from the ones available to help your analysis
- Visualize everything!

In [ ]:
# Plot a heatmap on all '_use' columns  and find the correlation values
drug_use = df_drug[[x for x in df_drug.columns if x.find('_use')>0]]

plt.figure(figsize=(14,14))
drug_use_corr = drug_use.corr()
sns.heatmap(drug_use_corr, annot=True)

In [ ]:
# Find the highest absolute correlation and corresponding pair
corrlen   = len(drug_use_corr)
corrmax   = max([max([0 if x==1 else np.abs(x) for x in drug_use_corr[r]]) for r in drug_use_corr])
drugspair = [drug for drug in [drug_use_corr.columns[n] if (corrmax in [np.abs(x) for x in drug_use_corr.iloc[n] if x<1]) else '' for n in range(corrlen)] if len(drug)>0]
print(f"Which 2 variables has the highest absolute correlation of:\nvalue = {corrmax}:\nwere : {drugspair}")


In [ ]:
# Ploting the stacked bar graph of the drug percentage use by age group
drug_use_df = df_drug[[x for x in df_drug.columns if (x=='age_group')|(x.find('_use')>0)]]
drug_use_df.plot(x= 'age_group', kind = 'bar', stacked =True,figsize=(40,28))
plt.xlabel('Age Group', fontsize = 35)
plt.ylabel('Percentage of people using drug',fontsize = 35)
plt.title = "Drug use disribution by Age-Group"
plt.legend(loc='upper left',fontsize=40)
plt.show()

In [ ]:
# Plotting the heatmap to display the correlation between the meadian number of times a users in an age group
drug_mean = df_drug[[x for x in df_drug.columns if x.find('_mean')>0]]

plt.figure(figsize=(14,14))
drug_mean_corr = drug_mean.corr()
sns.heatmap(drug_mean_corr, annot=True)

In [ ]:
# Find the highest absolute correlation and corresponding pair
corrlen   = len(drug_mean_corr)
corrmax   = max([max([0 if x==1 else np.abs(x) for x in drug_mean_corr[r]]) for r in drug_mean_corr])
drugspair = [drug for drug in [drug_mean_corr.columns[n] if (corrmax in [np.abs(x) for x in drug_mean_corr.iloc[n] if x<1]) else '' for n in range(corrlen)] if len(drug)>0]
print(f"Which 2 variables has the highest absolute correlation of:\nvalue = {corrmax}:\nwere : {drugspair}")


In [ ]:
# Find the next highest absolute correlation and corresponding pair
corrmax2  = max([max([0 if x>=corrmax else np.abs(x) for x in drug_mean_corr[r]]) for r in drug_mean_corr])
drugspair = [drug for drug in [drug_mean_corr.columns[n] if (corrmax2 in [np.abs(x) for x in drug_mean_corr.iloc[n] if x<1]) else '' for n in range(corrlen)] if len(drug)>0]
print(f"Which 2 variables has the next highest absolute correlation of:\nvalue = {corrmax2}:\nwere : {drugspair}")

In [ ]:
# work out a hypothese over hallucinogen and inhalant instead of cocaine and crack

### 7.3 Create a testable hypothesis about this data

Requirements for the question:

1. Write a specific question you would like to answer with the data (that can be accomplished with EDA).
2. Write a description of the "deliverables": what will you report after testing/examining your hypothesis?
3. Use EDA techniques of your choice, numeric and/or visual, to look into your question.
4. Write up your report on what you have found regarding the hypothesis about the data you came up with.


Your hypothesis could be on:

- Difference of group means
- Correlations between variables
- Anything else you think is interesting, testable, and meaningful!

**Important notes:**

You should be only doing EDA _relevant to your question_ here. It is easy to go down rabbit holes trying to look at every facet of your data, and so we want you to get in the practice of specifying a hypothesis you are interested in first and scoping your work to specifically answer that question.

Some of you may want to jump ahead to "modeling" data to answer your question. This is a topic addressed in the next project and **you should not do this for this project.** We specifically want you to not do modeling to emphasize the importance of performing EDA _before_ you jump to statistical analysis.

In [ ]:
# Plot a line graph to see both figures
hallucinogen = df_drug.hallucinogen_mean
inhalant = df_drug.inhalant_mean
cnt=df_drug['hallucinogen_mean'].count()
x = np.arange(cnt)
fig = plt.figure(1, figsize=(12, 8))
ax = plt.subplot(111)
ax.plot(x, df_drug['hallucinogen_mean'], label='hallucinogen_')
ax.plot(x, df_drug['inhalant_mean'], label='inhalant')
ax.set_title("Hallucinogen vs Inhalant", fontsize=35)
ax.set_xlabel("Record Number", fontsize=25)
ax.set_ylabel("Mean value", fontsize=25)
plt.show(ax)

In [ ]:
# calculate the pearson correlation coefficient between frequency of hallucinogen use and heroin use
r = stats.pearsonr(hallucinogen, inhalant)[0]
print(r, corrmax2)

# it matches the value in the matrix from pearson perpecive

In [ ]:
# generate 1000 random from the given data 'x'
def bootstrap_r(x, y, iterations=1000):
    boot_r = []
    inds = list(range(len(x)))
    for i in range(iterations):
        boot_inds = np.random.choice(inds, replace=True, size=len(inds))
        x_b = x[boot_inds]
        y_b = y[boot_inds]
        boot_r.append(stats.pearsonr(x_b, y_b)[0])
    return boot_r

In [ ]:
r_boots = bootstrap_r(hallucinogen, inhalant)

# 95%
lower = stats.scoreatpercentile(r_boots, 2.5)
upper = stats.scoreatpercentile(r_boots, 97.5)

print(lower, r, upper)

In [ ]:
r_boots = bootstrap_r(hallucinogen, inhalant)

# 99%
lower = stats.scoreatpercentile(r_boots, 0.5)
upper = stats.scoreatpercentile(r_boots, 99.5)

print(lower, r, upper)

In [ ]:
# Plot a Seaborn Distplot based on bootstrap
ax=sns.distplot(r_boots)
ax.set_xlabel("Percentile", fontsize=25)
ax.set_ylabel("Score", fontsize=25)

ax.set_title("Hallucinogen & Inhalant Percentile Score",fontsize=20)

In [ ]:
# Using T-statistics to compute P-Value
# Make a decision to accept or reject Null Hypothesis
alpha = 0.05

boot_mean = np.mean(r_boots)
boot_median = np.median(r_boots)

t_statistic = (boot_mean - boot_median)/(np.std(r_boots, ddof=1)/len(r_boots)**0.5)

## Find p-value.
p_value = t.sf(np.abs(t_statistic), len(r_boots))
print(f"P value is {p_value}")

if p_value < 0.5:
    print("Conclusion : Null Hypothesis H0 can be rejected.")
else:
    print("Conclusion : Null Hypothesis H0 can be accepted.")


In [ ]:
df1 = df_drug[['age','hallucinogen_mean','inhalant_mean']]
ax=df1.plot(x= 'age', kind = 'bar', stacked =False,figsize=(16,10))
plt.xlabel('Age', fontsize = 20)
plt.ylabel('Median number of people using alcohol and inhalant',fontsize = 20)
ax.set_title("Hallucinogen & Inhalant by Age",fontsize=35)

**Report**

95% of the correlation coefficient values fall into the range 
from 
    0.5932094733684674  
to  
    0.9196997408994209

99% of the correlation coeeficient values fall into the range 
from 
    0.33617842128346825 
to  
    0.9370982539051154

As the P-value 1.0978138352132973e-14 < 0.5, it can be concluded that the correlation between the frequency of the hallucinogen use and the heroin use is significant regardless the age group. Reject H0 Null Hyphothesis.



<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Introduction to dealing with outliers

---

Outliers are an interesting problem in statistics, in that there is not an agreed upon best way to define them. Subjectivity in selecting and analyzing data is a problem that will recur throughout the course.

1. Pull out the rate variable from the sat dataset.
2. Are there outliers in the dataset? Define, in words, how you _numerically define outliers._
3. Print out the outliers in the dataset.
4. Remove the outliers from the dataset.
5. Compare the mean, median, and standard deviation of the "cleaned" data without outliers to the original. What is different about them and why?

In [ ]:
df_rate = df_sat[['Rate']]
df_rate.boxplot()

In [ ]:
# compute the statistical values on the Rate data
rate_desc=df_rate.describe().T
rate_max=rate_desc['max'][0]
rate_min=rate_desc['min'][0]
rate_avg=rate_desc['mean'][0]
rate_std=rate_desc['std'][0]

rate_desc

In [ ]:
# drop/delete rows via record number
def del_rows(df, df_filter):
    df.drop([k for k in df_filter.keys()],inplace=True)
    return

In [ ]:
#An outlier is something that is above or below 1.5x std from the mean

rate_outlier_high = rate_avg + rate_std * 1.5
rate_outlier_low = rate_avg - rate_std  * 1.5

print(f"upper bound = {rate_outlier_high}, lower bound = {rate_outlier_low}\n")
print("Outlier data point if exceed the upper bound from mean:")
if rate_max >= rate_outlier_high:
    print("Found as following and will be removed:")
    df1 = df_rate['Rate'][df_rate['Rate'] >= rate_outlier_high]
    print(df1)
    # remove them
    del_rows(df_rate, df1)
else:
    print("does not exists")
    
print("Outlier data point if exceed the lower bound from mean:")
if rate_min <= rate_outlier_low:
    print("Found as following and will be removed:")
    df1 = df_rate['Rate'][df_rate['Rate'] <= rate_outlier_low]
    print(df1)
    # remove them
    del_rows(df_rate, df1)
else:
    print("does not exists ")    


In [ ]:
# Plot a seabon distplot graph
# input :
#        "data" - data series
#        "title" - Title of the graph
#        "labelx" - Label of the X axis
#        "labely" - Label of the Y axis
def univariate_analysis(data, title):
    print('mean: {}'.format(np.mean(data)))
    print('median: {}'.format(np.median(data)))
    print('std: {}'.format(np.std(data)))
    print('Percentiles:')
    for p in range(0,100+1,10):
        print('{}) {}'.format(p, np.percentile(data, p)))
        
    data = data - np.mean(data)
    
    print('Percentiles (after centering):')
    for p in range(0,100+1,10):
        print('{}) {}'.format(p, np.percentile(data, p)))
        
    sns.distplot(data)
    ax.set_title(title, fontsize=35)
    ax.set_xlabel("drugs values", fontsize=25)
    ax.set_ylabel("deviation", fontsize=25)

    plt.show()
    

In [ ]:
univariate_analysis(df_sat.Rate,"Original Data")

In [ ]:
univariate_analysis(df_rate.Rate, "Cleaned Data")

In [ ]:
mean_original = df_sat.Rate.mean()
median_original = df_sat.Rate.median()
std_original = df_sat.Rate.std()

mean_clean = df_rate.mean()
median_clean = df_rate.median()
std_clean = df_rate.std()[0]

print("\toriginal data \t\tvs cleaned data ")
print(f"mean : {mean_original} \t\t\tvs {mean_clean[0]}")
print(f"median : {median_original} \t\t\tvs {median_clean[0]}")
print(f"std : {std_original} \tvs {std_clean}")


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 9. Percentile scoring and spearman rank correlation

---

### 9.1 Calculate the spearman correlation of sat `Verbal` and `Math`

1. How does the spearman correlation compare to the pearson correlation? 
2. Describe clearly in words the process of calculating the spearman rank correlation.
  - Hint: the word "rank" is in the name of the process for a reason!


9.1.1
Spearman is a non-parametric test and Pearson is a parametric one.

Spearman is for ordinal kind of data like "Ranking" and 
Pearson is for ratio scale kind of data.

9.1.2

Objective :
Find the ranks in the two data and compute the Spearman rank correlation.

Step 1: 
Find the ranks for each individual data, 
such as order the values from greatest to smallest.
Assign the rank 1 to the highest score, 2 to the next highest and so on.

Step 2: 
Add a new column to the data as the difference between ranks. 
Add another column, square the values from previous.

Step 4: 
Sum all of the squared values from the last above mentioned column.

Step 5: 
Insert the values into the formula. 


### $$ \text{pearson correlation}\;r = cor(X, Y) =\frac{cov(X, Y)}{std(X)std(Y)}$$

The direct Python code :

    numpy.corrcoef(var1,var2)

### 9.2 Percentile scoring

Look up percentile scoring of data. In other words, the conversion of numeric data to their equivalent percentile scores.

http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.percentile.html

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html

1. Convert `Rate` to percentiles in the sat scores as a new column.
2. Show the percentile of California in `Rate`.
3. How is percentile related to the spearman rank correlation?

In [ ]:
df_sat['PCT'] = df_sat['Rate'].apply(lambda x: np.percentile(df_sat.Rate,x))

In [ ]:
df_sat.sort_values("PCT").tail()

In [ ]:
print(f"The percentile for state='CA' is {list(df_sat[(df_sat.State == 'CA')].PCT)[0]}\n")
print(df_sat[(df_sat.State == 'CA')])


**9.2.3. How is percentile related to the spearman rank correlation?**

The percentile score of a data point is equal to its rank divided by the number of data points. 

Below a graph with "Ranking" added, in compare with the percentile.

In [ ]:
df_sat['Rank'] = df_sat['PCT'].rank(ascending=1)

list_rank = list(df_sat['Rank'])
list_pct = list(df_sat['PCT'])
cnt_rank = len(list_rank)
range_rank = [x for x in range(1,1+cnt_rank)]

plt.plot(range_rank,list_rank, linewidth=2,c='red')
plt.plot(range_rank,list_pct, linewidth=3,c='blue')
plt.xlabel("Record Number")
plt.ylabel("Ranks & Percentile")
plt.show()


### 9.3 Percentiles and outliers

1. Why might percentile scoring be useful for dealing with outliers?
2. Plot the distribution of a variable of your choice from the drug use dataset.
3. Plot the same variable but percentile scored.
4. Describe the effect, visually, of coverting raw scores to percentile.

In [ ]:
def PlotDist(data, title, showpercentile):
    fig = plt.figure(figsize=(12,8))
    ax = fig.gca()   
    var_mean=np.mean(data)
    var_median=np.median(data)
    var_std=np.std(data)
    data = data - np.mean(data)    
    var_xtickers=[p for p in range(0,100+1,10)]
    var_percentile=[np.percentile(data, p) for p in range(0,100+1,10)]
    sns.distplot(data, kde=True, hist=False)
    plt.xlabel("drugs values",fontsize=25)
    plt.ylabel("deviation",fontsize=25)
    if showpercentile==True:
        for p in var_xtickers:
            label_pct=f"{p}%"
            pct=np.percentile(data, p)
            ax.axvline(pct, ls='dashed', lw=3, color='#333333', alpha=0.7)
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), round(p.get_height(),2), 
                fontsize=15, color='black', ha='center', va='bottom')
    plt.show()
    return 

In [ ]:
PlotDist(df_drug.hallucinogen_use,"Distribution of hallucinogen",False)

In [ ]:
PlotDist(df_drug.inhalant_use,"Distribution of inhalant",False)

In [ ]:
PlotDist(df_drug.hallucinogen_use,"Distribution of hallucinogen",True)

In [ ]:
PlotDist(df_drug.inhalant_use,"Distribution of inhalant",True)